In [3]:
import pandas as pd
import warnings


warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 65)
pd.set_option('display.max_rows', 100)


#these first few rows delete columns that are not necessary
grades = pd.read_csv('data_set_10_17.csv')
grades = grades.drop(['LEVEL'],axis='columns')
grades = grades.drop(['OFFER_DEPT_ABRVN'],axis='columns')
grades = grades.drop(['MAJOR_COLL'],axis='columns')
grades = grades.drop(['DEG_DESCR'], axis='columns')
grades = grades.drop(['DEG_DATEDB'], axis='columns')
grades = grades.drop(['Workday Enrolled in Fall 2024'], axis='columns')
grades = grades.drop(['Workday Enrolled in Fall 2024 Class Standing'], axis='columns')
grades = grades.drop(['Workday Enrolled in Fall 2024 Primary Program of Study'], axis='columns')
grades = grades.drop(['MAJOR_CURR'], axis='columns')
grades = grades.drop(['MAJOR_CURR.2'], axis='columns')
grades = grades.drop(['MAJOR_CURR.1'], axis='columns')
grades = grades.drop(['CLSFN_YR.1'], axis='columns')
grades = grades.drop(['LAST_RGST_TERM'], axis='columns')
grades = grades.drop(['SEM_CD.1'], axis='columns')
grades = grades.drop(['SECT'], axis='columns')
grades = grades.drop(['CRSE'], axis='columns')
grades = grades.drop(['SEM_CD'], axis='columns')
grades = grades.drop(['SEM_CCYY'], axis='columns')
grades = grades.drop(['ENTRY_SEM_CD'], axis='columns')
grades = grades.drop(['CLSFN_YR'], axis='columns')
grades = grades.drop(['MAJOR_COLL.1'], axis='columns')




#we are counting how many people recieved degrees
degrees = grades['DEG_CD'].value_counts()

#locating the rows that have graduate degrees. We don't want to include these.
mre = grades.loc[grades['DEG_CD'] == 'MRE']
mcp = grades.loc[grades['DEG_CD'] == 'MCP']
med = grades.loc[grades['DEG_CD'] == 'MED']
mfa = grades.loc[grades['DEG_CD'] == 'MFA']
mtr = grades.loc[grades['DEG_CD'] == 'MTR']
mhc = grades.loc[grades['DEG_CD'] == 'MHC']
mfs = grades.loc[grades['DEG_CD'] == 'MFS']
mho = grades.loc[grades['DEG_CD'] == 'MHO']
maa = grades.loc[grades['DEG_CD'] == 'MAA']
dvm = grades.loc[grades['DEG_CD'] == 'DVM']
mbs = grades.loc[grades['DEG_CD'] == 'MBS']
mba = grades.loc[grades['DEG_CD'] == 'MBA']
ma = grades.loc[grades['DEG_CD'] == 'MA']
men = grades.loc[grades['DEG_CD'] == 'MEN']
mfn = grades.loc[grades['DEG_CD'] == 'MFN']
mat = grades.loc[grades['DEG_CD'] == 'MAT']
phd = grades.loc[grades['DEG_CD'] == 'PHD']
ms = grades.loc[grades['DEG_CD'] == 'MS']

#dropped degrees are graduate degrees. kept degrees are undergraduate degrees.
dropped_degrees = ['MRE', 'MCP', 'MED', 'MFA', 'MTR', 'MHC', 'MFS', 'MHO', 'MAA', 'DVM', 'MBS', 'MBA', 'MA', 'MEN', 'MFN', 'MAT', 'PHD', 'MS']
dropped_degrees_lowercase = [mre, mcp, med, mfa, mtr, mhc, mfs, mho, maa, dvm, mbs, mba, ma, men, mfn, mat, phd, ms]
kept_degrees = ['BS', 'BA', 'BLS', 'BM', 'BFA', 'BID', 'BLA', 'BBA', 'BAR', 'BSN']


#this is me double checking that when we remove the dropped degrees we get the expected amount of rows
#dropped_lengths = [len(x) for x in dropped_degrees_lowercase]
#dropped_lengths
#totaldrop = sum(dropped_lengths)
#new_length = len(grades) - totaldrop

#here I drop the graduate degrees
for x in dropped_degrees:
    grades = grades.drop(grades[grades['DEG_CD'] == x].index)




#this replaces all bachelor's degrees with 1 and all non-graduated with 0
grades['DEG_CD'] = grades['DEG_CD'].fillna(0)
for x in kept_degrees:
    grades['DEG_CD'] = grades['DEG_CD'].replace(x,1)
grades

#this replaces all "Above C-" with a 3, "Satisfactory/Pass" with a 2 and "C- or Below" with a 1 
grades['GRADE_CATGORY'] = grades['GRADE_CATGORY'].replace('C- OR BELOW', 1)
grades['GRADE_CATGORY'] = grades['GRADE_CATGORY'].replace('Satisfactory/Pass', 2)
grades['GRADE_CATGORY'] = grades['GRADE_CATGORY'].replace('ABOVE C-', 3)
grades['ENTRY_CCYY'].value_counts()

grades[['ENTRY_CCYY', 'DEG_CD']].value_counts()

#We will only consider students who initially enrolled between 2011 and 2020 inclusive. 
#The reason for this is that 2020 enrollment is the latest that allows for 8 semesters (4 yrs) of courses to be taken
#and earlier than 2011 data may have issues ie courses that are no longer offered, people that got a degree and returned years laters
grades = grades[grades['ENTRY_CCYY'] <= 2020]
grades = grades[grades['ENTRY_CCYY'] >= 2011]

#There are courses with fewer than 50 people over the past ten years. These are removed.
min_enrollment = 50  

#Counting how many instances of each course we have
value_counts = grades['CRSE_TITLE'].value_counts()

#We are determining the criteria for removing a course from our data.
to_remove = value_counts[value_counts <= min_enrollment].index

#and now remove it
grades = grades[~grades.CRSE_TITLE.isin(to_remove)]


#We add a column that indicate how many years it took the student to graduate. NG if they did not.
#note: There is one student (4939) that graduated the same semester they enrolled, hence NG instead of 0.
grades['GRAD_TIME'] = ''
grades['GRAD_TIME'] = grades['SEM_CCYY.1'] - grades['ENTRY_CCYY']
grades['GRAD_TIME'] = grades['GRAD_TIME'].fillna('NG')

#We add one column for each course and temporarily stick a 0 in each column
courses = grades['CRSE_TITLE'].unique()

for x in courses:
    grades[x] =''
    grades[x] = grades[x].replace('',0)
    

#Now, in each row, if a student has taken a course, the corresponding column's 0 becomes a 1/2/3 depending on their grade.
for x in courses:
    grades[x][grades['CRSE_TITLE'] == x] = grades['GRADE_CATGORY']
    
#Then we remove the Grade Category and Course Title columns since they are no longer needed
grades = grades.drop(['GRADE_CATGORY'], axis='columns')
grades = grades.drop(['CRSE_TITLE'], axis='columns')

#Example student just to see things were working right.
grades.loc[grades['STUDENT'] == 13039]





#938 students have the same course listed more than once, either because they graduated with more than one degree, or because
#they took the course twice. We chose to take their highest grade in all these cases.
#The code below finds those students and keeps their highest grades.
#Students who took a course only one time will be unaffected.
#The information that is kept is: Student number, Entry Year, GRAD_TIME, If graduated (maybe redundant), All the course columns
grades = grades.groupby([grades.STUDENT]).max().reset_index()


#grades.to_csv('newdata.csv', index=False)


